In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

import re
import pandas as pd
import xlrd

In [3]:
result_3 = pd.read_excel('2020.03.01-2020.03.31_result.xlsx')

In [4]:
result_3.head()

,Unnamed: 0,years,company,title,contents,link
0,0,2020.03.31.,뉴시스,완도 유흥주점 84곳 자진휴업…사회적 거리두기 확산,100% 자진휴업 손님 출입 차단[완도=뉴시스] 완도군청 전경. (사진=뉴시스 DB...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,2020.03.31.,스포츠서울,"완도군, 관내 유흥주점 사회적 거리두기 동참",사회적 거리두기에 동참한 유흥 업소에 대한 감사 인사 현수막을 설치한 모습.(제공=...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,2020.03.31.,PRESSian,"완도군 유흥주점 84개소 사회적 거리두기 동참, 오는 4월 5일까지 휴업","100% 자진 휴업, 손님 출입 차단 등 방침 적극 이행 중[최영남 기자(=완도)(...",https://news.naver.com/main/read.nhn?mode=LSD&...
3,3,2020.03.31.,연합뉴스,사회적 거리두기 지쳤다…이젠 생활방역 지침 구체화해야,내달 5일 종료 시점 얼마 안 남아…실질적 지침 마련 시점사회적 거리두기 권고에 주...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,4,2020.03.31.,아시아경제신문,'사회적 거리두기'가 만든 '홀로' 新쇼핑,코로나가 바꾼 쇼핑트렌드건강용품 식품 판매 늘고대용량 식품 비축해 놓고재택 맞춤 상...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
del result_3['Unnamed: 0']

In [19]:
result_3 = result_3[result_3['link'].notnull()]

In [15]:
url="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=018&aid=0004300484&date=20190128&type=1&rankingSeq=5&rankingSectionId=101"    

header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36", 
    "referer": url,  
}

In [52]:
# link url 리스트
url = []
for l in result_3['link']:    
    url.append(l)

# link url 파싱
oid = []
aid = []
for i in url:
    oid.append(i.split("oid=")[1].split("&")[0]) 

for j in url:
    aid.append(j.split("aid=")[1])
        
# result_path
RESULT_PATH = '/home/jeehyun.lee.conda/'

c_List=[] 
d_List=[]

In [ ]:
def contents_flat(c_List,d_List): 
    flatList = [] 
    d_flat
    
    List=[]
    #f = open("C:/Users/User/Desktop/python study/beautifulSoup_ws/crawling_result/comment_text.txt", 'w', encoding='utf-8')
    for elem in c_List: 
        # if an element of a list is a list 
        # iterate over this list and add elements to flatList  
        
        #elem=re.sub('"',' ',str(elem))
        if type(elem) == list: 
            for e in elem: 
                flatList.append(e) 
        else: 
            flatList.append(elem) 
           
    for elem in d_List: 
        if type(elem) == list: 
            for e in elem: 
                e=re.sub('\\+0900',' ',e)
                d_flatList.append(e) 
        else: 
            elem=re.sub('\\+0900',' ',elem)
            d_flatList.append(elem) 

    #f.write(str(flatList))
    excel_make(flatList,d_flatList)
             
def excel_make(flatList,d_flatList): 
    result= {"comments":flatList, "date":d_flatList}
    df = pd.DataFrame.from_dict(result, orient = 'index')  # df로 변환
    df = df.T
    df.to_excel(RESULT_PATH+"comments_result_3.xlsx", sheet_name = 'sheet1')

def main():
    for o,a in [(oid, aid)]:
        for i in range(len(url)):
            page = 1
            comment_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+o[i]+"%2C"+a[i]+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE"
            print(comment_url)
            r=requests.get(comment_url,headers=header)
            cont=bs(r.content,"html.parser")     
            total_comm=str(cont).split('comment":')[1].split(",")[0]
            print(total_comm)
            
            match=re.findall('"contents":"([^\*]*)","userIdNo"', str(cont))   #소괄호 안에 있는것을 크롤링해온다.
            date=re.findall('"modTime":"([^\*]*)","modTimeGmt"', str(cont))

            c_List.append(match)
            d_List.append(date)
            
            while True:
                if int(total_comm) <= ((page) * 20): 
                    break
                else:
                    page += 1
                    
        print(len(c_List))
        print(len(d_List))
 
    contents_flat(c_List,d_List)  
    
main()

https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news003%2C0009786818&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
3
https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news468%2C0000642892&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
0
https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news002%2C0002128543&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&refresh=false&sort=FAVORITE
0
https://apis.naver.com/commentB